In [4]:
!pip install MetaTrader5
!pip install pandas_datareader
import MetaTrader5 as mt5
from datetime import datetime
import time
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt

In [6]:
if not mt5.initialize():
    mt5.shutdown()

In [8]:
def get_ohlc(symbol, timeframe, n=5):
    symbol = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    symbol = pd.DataFrame(symbol)
    symbol['time'] = pd.to_datetime(symbol['time'], unit='s')
    symbol['Pivot'] = (symbol['high'] + symbol['low'] + symbol['close'])/3
    symbol['R1'] = 2*symbol['Pivot'] - symbol['low']
    symbol['S1'] = 2*symbol['Pivot'] - symbol['high']
    symbol['R2'] = symbol['Pivot'] + (symbol['high'] - symbol['low'])
    symbol['S2'] = symbol['Pivot'] - (symbol['high'] - symbol['low'])
    symbol['R3'] = symbol['Pivot'] + 2*(symbol['high'] - symbol['low'])
    symbol['S3'] = symbol['Pivot'] - 2*(symbol['high'] - symbol['low'])
    
    symbol.set_index('time', inplace=True)
    return symbol

In [9]:
symbol = get_ohlc('MGLU3', mt5.TIMEFRAME_M5)

In [11]:
symbol

,open,high,low,close,tick_volume,spread,real_volume,Pivot,R1,S1,R2,S2,R3,S3
time,,,,,,,,,,,,,,
2021-06-02 15:45:00,20.39,20.43,20.33,20.34,308,1,806100,20.366667,20.403333,20.303333,20.466667,20.266667,20.566667,20.166667
2021-06-02 15:50:00,20.34,20.35,20.27,20.34,366,1,520300,20.320000,20.370000,20.290000,20.400000,20.240000,20.480000,20.160000
2021-06-02 15:55:00,20.34,20.41,20.34,20.36,249,1,254400,20.370000,20.400000,20.330000,20.440000,20.300000,20.510000,20.230000
2021-06-02 16:00:00,20.36,20.39,20.32,20.38,200,1,273500,20.363333,20.406667,20.336667,20.433333,20.293333,20.503333,20.223333
2021-06-02 16:05:00,20.38,20.38,20.26,20.27,261,1,440800,20.303333,20.346667,20.226667,20.423333,20.183333,20.543333,20.063333


In [44]:
ticket = 'WINM21' 

volume = 10.0
point = mt5.symbol_info(ticket).point
price = mt5.symbol_info(ticket).ask
deviation = 0

request_buy = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": ticket,
    "volume": volume,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    "sl": price - 100*point,
    "tp": price + 100*point,
    "magic": 6667771,
    "deviation": deviation,
    "comment": "Abrindo ordem de compra",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}

In [45]:
volume = 10.0
point = mt5.symbol_info(ticket).point
price = mt5.symbol_info(ticket).bid
deviation = 0

request_sell = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": ticket,
    "volume": volume,
    "type": mt5.ORDER_TYPE_SELL,
    "price": price,
    "sl": price + 100*point,
    "tp": price - 100*point,
    "magic": 6667771,
    "deviation": deviation,
    "comment": "Abrindo ordem de venda",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}

In [ ]:
while True:
    ativo = get_ohlc(ticket, mt5.TIMEFRAME_M1, 10)
    tick = mt5.symbol_info_tick(ticket)
    
    print(f'{ticket}, last={tick.last}, topo_compra={tick.bid}, topo_venda={tick.ask}, R1={ativo.R1[-1 -1]}, S1={ativo.S1[-1 -1]}', end='\r')
    
    if tick.last > ativo['R1'][-1 -1]:
        if mt5.positions_get(symbol=ticket) == () or mt5.positions_get(symbol=ticket)[0][10] == 1:
            result = mt5.order_send(request_buy)
            print(f'Ordem de compra enviada: {volume} de {ticket} ao preço de {price}', end='\r')
        
    if tick.last < ativo['S1'][-1 -1]:
        if mt5.positions_get(symbol=ticket) == () or mt5.positions_get(symbol=ticket)[0][10] == 1:
            result = mt5.order_send(request_sell)
            print(f'Ordem de venda enviada: {volume} de {ticket} ao preço de {price}', end='\r')
            
    time.sleep(0.5)
        